In [681]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys

%matplotlib inline

# DataFrame

## Create DataFrame

pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)

data : numpy ndarray (structured or homogeneous), dict, or DataFrame. Dict can contain Series, arrays, constants, or list-like objects

index : Index or array-like Index to use for resulting frame. Will default to RangeIndex if no indexing information part of input data and no index provided

dtype : dtype, default None. Data type to force. Only a single dtype is allowed. If None, infer

columns : Index or array-like Column labels to use for resulting frame. Will default to RangeIndex (0, 1, 2, ..., n) if no column labels are provided

### From Dictionary

In [682]:
d = {'col1': [1, 2], 'col2': [3, 4]}
df = pd.DataFrame(data=d, index=['a', 'b'], dtype=np.int8)
df

,col1,col2
a,1,3
b,2,4


In [683]:
df.dtypes # If None, inferred dtype is int64.

col1    int8
col2    int8
dtype: object

### From numpy ndarray

In [684]:
df2 = pd.DataFrame(np.random.randint(low=0, high=10, size=(10, 5)), columns=['a', 'b', 'c', 'd', 'e'])
df2

,a,b,c,d,e
0,7,5,5,2,2
1,7,8,4,1,9
2,8,3,5,5,7
3,7,1,5,9,0
4,2,3,0,7,3
5,5,1,0,8,2
6,5,6,5,9,4
7,7,5,2,4,3
8,3,8,5,8,6
9,9,8,1,0,0


### From list

In [685]:
data = [[12, 2], [0, 4], [10, 20], [1, 4], [7, 1], [16, 36]] # data list
tuples = [('cobra', 'mark i'), ('cobra', 'mark ii'),('sidewinder', 'mark i'), 
          ('sidewinder', 'mark ii'),('viper', 'mark ii'), ('viper', 'mark iii')]
index = pd.MultiIndex.from_tuples(tuples)
index

MultiIndex(levels=[['cobra', 'sidewinder', 'viper'], ['mark i', 'mark ii', 'mark iii']],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 1, 2]])

In [686]:
df = pd.DataFrame(data, columns=['max_speed', 'sheild'], index=index)
df

max_speed  sheild
cobra      mark i           12       2
           mark ii           0       4
sidewinder mark i           10      20
           mark ii           1       4
viper      mark ii           7       1
           mark iii         16      36

## DataFrame Methods

In [687]:
d = list(range(10, 13))
df = pd.DataFrame(d)
df

,0
0,10
1,11
2,12


### column, index

In [688]:
df.columns

RangeIndex(start=0, stop=1, step=1)

In [689]:
df.columns = ['col1'] # rename column(s)
df

,col1
0,10
1,11
2,12


In [690]:
df.index

RangeIndex(start=0, stop=3, step=1)

In [691]:
df.index = ['a', 'b', 'c']
df

,col1
a,10
b,11
c,12


In [692]:
df_ = df.set_index(['col1'])
df_.index

Int64Index([10, 11, 12], dtype='int64', name='col1')

#### Add new columns

In [693]:
df['new_col'] = 13 # add new column. Can't use df.new_col
df

,col1,new_col
a,10,13
b,11,13
c,12,13


### Select 

#### Select vertical pieces

In [694]:
df.new_col # pandas.core.series.Series # Here we can use df.new_col

a    13
b    13
c    13
Name: new_col, dtype: int64

In [695]:
df[['new_col', 'col1']]

,new_col,col1
a,13,10
b,13,11
c,13,12


#### Select horizontal pieces

In [696]:
df.loc['a'] # pandas.core.series.Series

col1       10
new_col    13
Name: a, dtype: int64

In [697]:
df.loc['a':'b'] # df.loc[inclusive:inclusive]

,col1,new_col
a,10,13
b,11,13


#### Select both vertically and horizontally

In [698]:
df.loc['a', 'new_col'] # numpy.int64

13

In [699]:
df.loc['a', 'col1':'new_col'] # pandas.core.series.Series

col1       10
new_col    13
Name: a, dtype: int64

In [700]:
df.loc['a':'b', 'col1':'new_col'] # pandas.core.frame.DataFrame ## 'a':'b' can be replaced by df.index[0:2]

,col1,new_col
a,10,13
b,11,13


In [701]:
df.loc[['a','c'], ['col1', 'new_col']] # pandas.core.frame.DataFrame

,col1,new_col
a,10,13
c,12,13


#### Select by boolean array

In [702]:
df.loc[[True, True, False]] # select first two rows

,col1,new_col
a,10,13
b,11,13


In [703]:
df.loc[df['col1']>10] # select rows whose col1 > 10

,col1,new_col
b,11,13
c,12,13


In [704]:
df.loc[lambda df: df['col1']>10, ['new_col']] # select rows whose rol1>10 and show their new_col column

,new_col
b,13
c,13


### Stack, Unstack

In [705]:
stack = df.stack() # Bring the columns and place them in the index
stack

a  col1       10
   new_col    13
b  col1       11
   new_col    13
c  col1       12
   new_col    13
dtype: int64

In [706]:
stack.index # # The index now includes the column names

MultiIndex(levels=[['a', 'b', 'c'], ['col1', 'new_col']],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

In [707]:
unstack = df.unstack()
unstack

col1     a    10
         b    11
         c    12
new_col  a    13
         b    13
         c    13
dtype: int64

In [708]:
unstack.index

MultiIndex(levels=[['col1', 'new_col'], ['a', 'b', 'c']],
           labels=[[0, 0, 0, 1, 1, 1], [0, 1, 2, 0, 1, 2]])

In [709]:
df = stack.unstack()
df

,col1,new_col
a,10,13
b,11,13
c,12,13


In [710]:
df.T # exchange columns and indices

,a,b,c
col1,10,11,12
new_col,13,13,13


### groupby

In [711]:
d = {'num1':[1,1,2,2,3,3],
     'num2':[1,2,3,1,2,3],
     'letter':['a','a','a','b','b','b']}

df = pd.DataFrame(d)
df

,letter,num1,num2
0,a,1,1
1,a,1,2
2,a,2,3
3,b,2,1
4,b,3,2
5,b,3,3


In [712]:
letter = df.groupby(['letter']).sum()
letter

,num1,num2
letter,,
a,4,6
b,8,6


In [713]:
letter_num = df.groupby(['letter', 'num1']).sum()
letter_num

num2
letter num1      
a      1        3
       2        3
b      2        1
       3        5

In [714]:
letter_num.index

MultiIndex(levels=[['a', 'b'], [1, 2, 3]],
           labels=[[0, 0, 1, 1], [0, 1, 1, 2]],
           names=['letter', 'num1'])

In [715]:
letter_num = df.groupby(['letter', 'num1'], as_index=False).sum()
letter_num

,letter,num1,num2
0,a,1,3
1,a,2,3
2,b,2,1
3,b,3,5


In [716]:
letter_num.index

Int64Index([0, 1, 2, 3], dtype='int64')

# Application

## Calc for outliers

In [717]:
States = ['NY', 'NY', 'NY', 'NY', 'FL', 'FL', 'GA', 'GA', 'FL', 'FL'] 
data = [1.0, 2, 3, 4, 5, 6, 7, 8, 9, 10]
idx = pd.date_range('1/1/2012', periods=10, freq='MS')
df1 = pd.DataFrame(data, index=idx, columns=['Revenue'])
df1['State'] = States

data2 = [10.0, 10.0, 9, 9, 8, 8, 7, 7, 6, 6]
idx2 = pd.date_range('1/1/2013', periods=10, freq='MS')
df2 = pd.DataFrame(data2, index=idx2, columns=['Revenue'])
df2['State'] = States

df = pd.concat([df1, df2])
df

,Revenue,State
2012-01-01,1.0,NY
2012-02-01,2.0,NY
2012-03-01,3.0,NY
2012-04-01,4.0,NY
2012-05-01,5.0,FL
2012-06-01,6.0,FL
2012-07-01,7.0,GA
2012-08-01,8.0,GA
2012-09-01,9.0,FL
2012-10-01,10.0,FL


### Method 1: groupby.transform

In [718]:
newdf = df.copy()
StateMonth = newdf.groupby(['State', lambda x:x.month])
StateMonth.sum()

Revenue
State            
FL    5      13.0
      6      14.0
      9      15.0
      10     16.0
GA    7      14.0
      8      15.0
NY    1      11.0
      2      12.0
      3      12.0
      4      13.0

In [719]:
newdf['x-Mean'] = StateMonth.transform(lambda x: abs(x-x.mean()))
newdf['1.96*std'] = StateMonth.transform(lambda x: 1.96*x.std())
newdf['Outlier'] = StateMonth.transform(lambda x: abs(x-x.mean()) > 1.96*x.std())
newdf

,Revenue,State,x-Mean,1.96*std,Outlier
2012-01-01,1.0,NY,4.5,12.473364,False
2012-02-01,2.0,NY,4.0,11.087434,False
2012-03-01,3.0,NY,3.0,8.315576,False
2012-04-01,4.0,NY,2.5,6.929646,False
2012-05-01,5.0,FL,1.5,4.157788,False
2012-06-01,6.0,FL,1.0,2.771859,False
2012-07-01,7.0,GA,0.0,0.000000,False
2012-08-01,8.0,GA,0.5,1.385929,False
2012-09-01,9.0,FL,1.5,4.157788,False
2012-10-01,10.0,FL,2.0,5.543717,False


### Method 2: groupby.apply

In [720]:
newdf = df.copy()

StateMonth = newdf.groupby(['State', lambda x:x.month])

def s(group):
    group['x-Mean'] = abs(group['Revenue'] - group['Revenue'].mean())
    group['1.96*std'] = 1.96 * group['Revenue'].std()
    group['Outlier'] = group['x-Mean'] > group['1.96*std']
    return group

newdf2 = StateMonth.apply(s)
newdf2

,Revenue,State,x-Mean,1.96*std,Outlier
2012-01-01,1.0,NY,4.5,12.473364,False
2012-02-01,2.0,NY,4.0,11.087434,False
2012-03-01,3.0,NY,3.0,8.315576,False
2012-04-01,4.0,NY,2.5,6.929646,False
2012-05-01,5.0,FL,1.5,4.157788,False
2012-06-01,6.0,FL,1.0,2.771859,False
2012-07-01,7.0,GA,0.0,0.000000,False
2012-08-01,8.0,GA,0.5,1.385929,False
2012-09-01,9.0,FL,1.5,4.157788,False
2012-10-01,10.0,FL,2.0,5.543717,False


### Compare groupby.transform and groupby.apply

#### groupby.transform

In [721]:
df_ = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar','foo', 'bar'],
                   'B' : ['one', 'one', 'two', 'three','two', 'two'],
                   'C' : [1, 5, 5, 2, 5, 5],
                   'D' : [2.0, 5., 8., 1., 2., 9.]})
df_

,A,B,C,D
0,foo,one,1,2.0
1,bar,one,5,5.0
2,foo,two,5,8.0
3,bar,three,2,1.0
4,foo,two,5,2.0
5,bar,two,5,9.0


In [722]:
grouped = df_.groupby('A')
grouped.transform(lambda x: (x - x.mean()) / x.std()) # @1

,C,D
0,-1.154701,-0.577350
1,0.577350,0.000000
2,0.577350,1.154701
3,-1.154701,-1.000000
4,0.577350,-0.577350
5,0.577350,1.000000


@1: Call function producing a like-indexed Series on **each group** and return a Series having the same indexes as the original object filled with the transformed values

####  groupby.apply 

In [723]:
ser = pd.Series([0, 1, 2], index='a a b'.split())
ser

a    0
a    1
b    2
dtype: int64

In [724]:
ser.index

Index(['a', 'a', 'b'], dtype='object')

In [725]:
g = ser.groupby(ser.index)
g.apply(lambda x:  x*2 if x.name == 'b' else x/2) # @1

a    0.0
a    0.5
b    4.0
dtype: float64

@1: Apply function ``func``  group-wise and combine the results together.
The function passed to ``apply`` must take a series as its first
argument and return a dataframe, a series or a scalar. ``apply`` will
then take care of combining the results back together into a single
dataframe or series. ``apply`` is therefore a highly flexible
grouping method.

In my opinion, `transform` can get integral property of a group of numbers (such as mean, std, and anything), while `apply` can only do calc on individual number in a group.

### Method 3: Consider all data

In [726]:
newdf = df.copy()

newdf['x-Mean'] = abs(newdf['Revenue'] - newdf['Revenue'].mean())
newdf['1.96*std'] = 1.96*newdf['Revenue'].std()
newdf['Outlier'] = newdf['x-Mean'] > newdf['1.96*std']
newdf

,Revenue,State,x-Mean,1.96*std,Outlier
2012-01-01,1.0,NY,5.75,5.200273,True
2012-02-01,2.0,NY,4.75,5.200273,False
2012-03-01,3.0,NY,3.75,5.200273,False
2012-04-01,4.0,NY,2.75,5.200273,False
2012-05-01,5.0,FL,1.75,5.200273,False
2012-06-01,6.0,FL,0.75,5.200273,False
2012-07-01,7.0,GA,0.25,5.200273,False
2012-08-01,8.0,GA,1.25,5.200273,False
2012-09-01,9.0,FL,2.25,5.200273,False
2012-10-01,10.0,FL,3.25,5.200273,False


### If non-Gaussian distribution: 

In [727]:
# make a copy of original df
newdf = df.copy()

State = newdf.groupby('State')

newdf['Lower'] = State['Revenue'].transform(lambda x: x.quantile(q=.25) 
                                            - (1.5*(x.quantile(q=.75)-x.quantile(q=.25))) )
newdf['Upper'] = State['Revenue'].transform(lambda x: x.quantile(q=.75) 
                                            + (1.5*(x.quantile(q=.75)-x.quantile(q=.25))) )
newdf['Outlier'] = (newdf['Revenue'] < newdf['Lower']) | (newdf['Revenue'] > newdf['Upper']) 
newdf

,Revenue,State,Lower,Upper,Outlier
2012-01-01,1.0,NY,-7.000,19.000,False
2012-02-01,2.0,NY,-7.000,19.000,False
2012-03-01,3.0,NY,-7.000,19.000,False
2012-04-01,4.0,NY,-7.000,19.000,False
2012-05-01,5.0,FL,2.625,11.625,False
2012-06-01,6.0,FL,2.625,11.625,False
2012-07-01,7.0,GA,6.625,7.625,False
2012-08-01,8.0,GA,6.625,7.625,True
2012-09-01,9.0,FL,2.625,11.625,False
2012-10-01,10.0,FL,2.625,11.625,False


# Export Data

In [728]:
# df.to_csv('data.csv') # columns=None, header=True, index=True, index_label=None
# df.to_csv('data.txt') 
# df.to_excel('data.xls') # sheet_name='Sheet1', columns=None, header=True, index=True, 
                          # index_label=None, startrow=0, startcol=0
# df.to_json('data.json')

# Load Data

In [729]:
# pd.read_csv('data.csv') # header='infer', names=None, index_col=None
# pd.read_csv('data.txt') 
# pd.read_excel('data.xls') # sheet_name=0, header=0, names=None, index_col=None
# pd.read_json('data.json') 